## Iteration - 2

This iteration trains a 2d U-net on all the 108 Uchicago scans. 

In [1]:
from google.colab import drive
drive.mount("/content/drive")

import os
import pandas as pd
import datetime
import matplotlib.pyplot as plt
import numpy as np

Mounted at /content/drive


In [ ]:
os.chdir('/content/drive/MyDrive/01_eom_segmentation/02_training/03_2dunet_108uc_coronal/')
os.getcwd()

In [ ]:
#%tensorflow_version 1.x
import tensorflow
print(tensorflow.__version__)

import keras

!python --version

print(tensorflow.__version__)
print(keras.__version__)

# Load the TensorBoard notebook extension
%load_ext tensorboard

2.4.0
Python 3.6.9
2.4.0
2.4.3


In [3]:
from helpers.data import *
from helpers.datagenerator import *
from helpers.model import *
from helpers.visualize import *

ModuleNotFoundError: ignored

In [ ]:
data_folder='../compiled_npys'

In [2]:
'''
Smaller batch size performs better - Tried with 200 batches but try with 20 maybe for better generalization
Higher patch size performs better - already tried with 64 patch size, try with 96 or 80 
Multi-class tends to perform better - try 1 class and 4 class - 1 class better than multi
One-Zero weights - balanced tends to converge faster - But 0.9 and 0.1 work well in training but don't generalize so well 
WBCE loss seems to work best with SGD with momentum - try other losses with other optimizers - there is one loss that combines wbce and dice too. 
SGD with Momentum seems to work best - other optimizers can be tried. For optimizers with adaptive learning rate, we need to use higher initial LRs, and for SGD we need to use lower learning rate. Nothing else apart from SGD with momentum worked
LR schedule - exponential decay works well, but you can also try ReduceLROnPlateau callback in keras
Weight initializers - GlorotUniform seems to work well. 

Cosine decay with warm restarts
Use pydensecrf to post-process output

'''


# Initialize Hyperparameters

# Image size
#IMG_SIZE=64                                         # multiple of 32 (2^depth - unet depth 5 will need atleast 32)
IMG_SIZE=80                                         # multiple of 32 (2^depth - unet depth 5 will need atleast 32)
#IMG_SIZE=96                                         # multiple of 32 (2^depth - unet depth 5 will need atleast 32)

# 3D vs 2D 
patch_sz=(IMG_SIZE, IMG_SIZE)

# 2D view-plane
view_plane_2d='2d'                                  # 'coronal', 'axial', 'sagittal', '3d'
muscles='all'                                       # 'all', 'mediallateral', 'superiorinferior'

# Set unet depth


unet_depth='na'

#n_classes=1                                         # 4 if all; 2 if mediallateral, superiorinferior
n_classes=4                                         # 4 if all; 2 if mediallateral, superiorinferior

num_scans_in_batch=4
#num_scans_in_batch=2

#num_patches_in_batch=200                            # keep below 20 for 3d patches and around 100-200 for 2D ; NEEDS TO BE A MULTIPLE OF 4
num_patches_in_batch=100                            # keep below 20 for 3d patches and around 100-200 for 2D 
#num_patches_in_batch=50                            # keep below 20 for 3d patches and around 100-200 for 2D 

n_filters_start = 'na'                                # Unet standard is 64, can try others


num_epochs=500
#num_epochs=2

#LR=3e-5
#LR=3e-4     # Check first for adam
LR=0.005   # This worked really well for SGD with momentum
#LR=0.05
#LR=0.1

# Windowing settings
window_center=0
window_width=400                                    # This is decided from intensity graphs

# Set loss function
weighted_binary_crossentropy = create_weighted_binary_crossentropy(zero_weight=0.3, one_weight=0.7)

#loss_function="focal"
#training_loss=focal_tversky
#loss_function="wbce"                                # "dice" or "focal" - use focal tversky loss; dice loss is non-convex. 
#training_loss=weighted_binary_crossentropy          # "dice_coef_loss" or "focal_tversky"
#loss_function="dice"                               # "dice" or "focal" - use focal tversky loss; dice loss is non-convex. 
#training_loss=dice_coef_loss                       # "dice_coef_loss" or "focal_tversky"
loss_function="dice_wbce_loss"                      # "dice" or "focal" - use focal tversky loss; dice loss is non-convex. 
training_loss=dice_wbce_loss                        # "dice_coef_loss" or "focal_tversky"


monitoring_loss=training_loss.__name__

# Model architecture
network='unet_2d'                                   # unet, fpn, linknet, pspnet
backbone='unet' #    'mobilenetv2'                  # https://github.com/qubvel/segmentation_models
model_architecture=network+'_'+backbone

print("In this iteration we are training a "+
      str(model_architecture)+" model with "+
      loss_function+" loss function to predict "+
      str(n_classes)+" "+
      muscles+" muscles on "+
      view_plane_2d+" slices")


NameError: ignored

In [ ]:
#unet=unet2d(input_size = (IMG_SIZE,IMG_SIZE,1), n_classes=n_classes)
unet=unet_2d_model_deep4(n_classes=n_classes, im_sz=IMG_SIZE, n_channels=1, n_filters_start=100, growth_factor=2, upconv=True)
unet.summary(line_length=150)

Model: "model"
______________________________________________________________________________________________________________________________________________________
Layer (type)                                     Output Shape                     Param #           Connected to                                      
input_1 (InputLayer)                             [(None, 80, 80, 1)]              0                                                                   
______________________________________________________________________________________________________________________________________________________
batch_normalization (BatchNormalization)         (None, 80, 80, 1)                4                 input_1[0][0]                                     
______________________________________________________________________________________________________________________________________________________
conv2d (Conv2D)                                  (None, 80, 80, 100)           

In [ ]:
series_directory =pd.read_excel('02_training_2dunet_coronal_108uc_dir.xlsx')

train_ids=series_directory[series_directory['Train_val_split'].str.contains("Train")]['DIR_ID'].to_list()
val_ids=series_directory[series_directory['Train_val_split'].str.contains("Val")]['DIR_ID'].to_list()

print(len(train_ids), len(val_ids))

training_scan_indexes = [train_id for train_id in train_ids if all( dim >= IMG_SIZE for dim in np.load(os.path.join(data_folder, 'SCAN_'+str(train_id)+'.npy')).shape)]
validation_scan_indexes = [val_id for val_id in val_ids if all( dim >= IMG_SIZE for dim in np.load(os.path.join(data_folder, 'SCAN_'+str(val_id)+'.npy')).shape)]

'''
# For overfitting
training_scan_indexes = training_scan_indexes[0:4]
validation_scan_indexes = validation_scan_indexes[0:4]
'''

print(len(training_scan_indexes),len(validation_scan_indexes))

71 37
58 28


In [ ]:
'''
from tensorflow.keras.experimental import CosineDecayRestarts
lr_decayed_fn = (CosineDecayRestarts(initial_learning_rate = LR, first_decay_steps = 2500))
LR=lr_decayed_fn
'''

#opt = Adam(learning_rate=LR)
#opt = Adam()
#opt = AdaBound(lr=LR, final_lr=0.1)
opt = SGD(learning_rate=LR, momentum=0.9, nesterov=True)
#opt = SGD(learning_rate=LR, momentum=0.9, nesterov=True)

#opt = Adadelta(learning_rate=0.001, rho=0.95, epsilon=1e-07, name="Adadelta")
#opt = RMSprop(learning_rate=LR, rho=0.9, momentum=0.9, epsilon=1e-07, centered=True)


unet.compile(optimizer=opt, loss=training_loss, metrics=[dice_coef_loss])

In [ ]:
training_params = {'TRAINING_DATA_DIR':data_folder, 
                   'scan_IDs':training_scan_indexes, 
                   'num_scans_in_batch' : num_scans_in_batch, 
                   'num_patches_in_batch':num_patches_in_batch, 
                   'dim':patch_sz, 
                   'view_plane':view_plane_2d,
                   'muscles':muscles,
                   'window_width':window_width,
                   'n_channels':1, 
                   'n_classes':n_classes, 
                   'shuffle':True,
                   'validation_flag':0}

validation_params = {'TRAINING_DATA_DIR':data_folder, 
                     'scan_IDs':validation_scan_indexes, 
                     'num_scans_in_batch' : training_params['num_scans_in_batch']*2, 
                     'num_patches_in_batch':num_patches_in_batch,
                     'dim':patch_sz, 
                     'view_plane':view_plane_2d,
                     'muscles':muscles,
                     'window_width':window_width,
                     'n_channels':1, 
                     'n_classes':n_classes, 
                     'shuffle':True,
                     'validation_flag':1}

In [ ]:
# Generators
training_generator = DataGenerator(**training_params)
validation_generator = DataGenerator(**validation_params)

In [ ]:
# The iteration with specified hyperparameters will be saved using this suffix
iteration_suffix=str(str(model_architecture)+
                     "_"+str(IMG_SIZE)+"sz"+ 
                     "_"+str(unet_depth)+"dp"+ 
                     "_"+str(len(patch_sz))+"d"+
                     "_"+str(view_plane_2d)+"vp"+
                     "_"+str(num_scans_in_batch)+"sb"+
                     "_"+str(num_patches_in_batch)+"pb"+
                     "_"+str(num_epochs)+"ep"+
                     "_"+str(n_filters_start)+"sf"+
                     "_"+str(LR)+"lr"+
                     "_"+str(window_width)+"ww"+
                     "_"+str(loss_function)+"ls"
                     )



results_dir=os.path.join('./', iteration_suffix)
log_dir = str(results_dir)+'/'+datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

if not os.path.exists(results_dir): os.mkdir(results_dir)
if not os.path.exists(log_dir): os.mkdir(log_dir)

print("Iteration suffix is: "+str(iteration_suffix))
print("Results directory: "+str(results_dir))
print("Log directory for Tensorboard: "+str(log_dir))

Iteration suffix is: unet_2d_unet_80sz_nadp_2d_2dvp_4sb_100pb_500ep_nasf_0.005lr_400ww_dice_wbce_lossls
Results directory: ./unet_2d_unet_80sz_nadp_2d_2dvp_4sb_100pb_500ep_nasf_0.005lr_400ww_dice_wbce_lossls
Log directory for Tensorboard: ./unet_2d_unet_80sz_nadp_2d_2dvp_4sb_100pb_500ep_nasf_0.005lr_400ww_dice_wbce_lossls/20210104-045933


In [ ]:
# Continue training for specific number of epochs
csv_logger = CSVLogger(os.path.join(log_dir, "01_training_history"+str(iteration_suffix)+".csv"), append=True)
weight_saver = ModelCheckpoint(os.path.join(log_dir, '02_unet'+str(iteration_suffix)+'.h5'), 
                               monitor='val_dice_coef_loss', 
                               save_best_only=True)

tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)
es = EarlyStopping(monitor='val_dice_coef_loss', mode='min', verbose=1, patience=50)

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=10, min_lr=0.00001)

In [ ]:
# Load the TensorBoard notebook extension
%load_ext tensorboard
%tensorboard --logdir './'

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6006 (pid 302), started 14:04:37 ago. (Use '!kill 302' to kill it.)

<IPython.core.display.Javascript object>

In [ ]:
# TRAIN

history =  unet.fit(x=training_generator, 
                    validation_data=validation_generator, 
                    epochs=num_epochs, 
                    verbose=1
                    , callbacks=[weight_saver, tensorboard_callback, reduce_lr 
                                 ] 
                    )

Epoch 1/500
55/55 [==============================] - 123s 2s/step - loss: 1.0786 - dice_coef_loss: 0.9882 - val_loss: 1.0580 - val_dice_coef_loss: 0.9856
Epoch 2/500
55/55 [==============================] - 133s 2s/step - loss: 1.0332 - dice_coef_loss: 0.9839 - val_loss: 1.0525 - val_dice_coef_loss: 0.9852
Epoch 3/500
55/55 [==============================] - 111s 2s/step - loss: 1.0109 - dice_coef_loss: 0.9806 - val_loss: 1.0414 - val_dice_coef_loss: 0.9847
Epoch 4/500
55/55 [==============================] - 109s 2s/step - loss: 0.9972 - dice_coef_loss: 0.9745 - val_loss: 1.0269 - val_dice_coef_loss: 0.9828
Epoch 5/500
55/55 [==============================] - 106s 2s/step - loss: 0.9821 - dice_coef_loss: 0.9641 - val_loss: 1.0094 - val_dice_coef_loss: 0.9797
Epoch 6/500
55/55 [==============================] - 102s 2s/step - loss: 0.9621 - dice_coef_loss: 0.9447 - val_loss: 0.9990 - val_dice_coef_loss: 0.9769
Epoch 7/500
55/55 [==============================] - 104s 2s/step - loss: 0.

In [ ]:
np.random.randint(100,101)